# 📘 Clase 21: Pydantic y Validacion

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/heldigard/unaula-IF0100-POO-II/blob/main/notebooks/unidad-03/clase-21-pydantic-validacion.ipynb)

## 🎯 Objetivos de Aprendizaje

Al finalizar esta clase, seras capaz de:
- Crear modelos con Pydantic
- Validar datos automaticamente
- Usar Field para validaciones avanzadas
- Crear modelos anidados
- Manejar conversion de tipos
- Usar Config para personalizar modelos

---

## 📚 Teoria: Pydantic

### ¿Que es Pydantic?

**Pydantic** es una biblioteca para validacion de datos usando type hints de Python.

**Caracteristicas:**
- ✅ Validacion automatica basada en tipos
- ✅ Conversion de tipos (coercion)
- ✅ Manejo de errores detallado
- ✅ Serializacion/deserializacion JSON
- ✅ Integracion nativa con FastAPI

---

## 📝 Modelos Basicos

In [ ]:
# ============================================
# MODELOS PYDANTIC BASICOS
# ============================================

from pydantic import BaseModel, Field
from typing import Optional, List
from datetime import datetime

# Modelo simple
class Usuario(BaseModel):
    id: int
    nombre: str
    email: str
    edad: Optional[int] = None

# Crear instancia
usuario = Usuario(id=1, nombre='Juan', email='juan@test.com')
print(f'Usuario: {usuario}')
print(f'JSON: {usuario.json()}')

# Validacion automatica
try:
    usuario_invalido = Usuario(id='abc', nombre='Ana', email='ana@test')
except Exception as e:
    print(f'\n❌ Error de validacion: {e}')

In [ ]:
# ============================================
# VALIDACIONES CON FIELD
# ============================================

from pydantic import Field, validator

class Producto(BaseModel):
    nombre: str = Field(..., min_length=3, max_length=50, description='Nombre del producto')
    precio: float = Field(..., gt=0, description='Precio debe ser mayor a 0')
    stock: int = Field(default=0, ge=0, description='Stock no puede ser negativo')
    categoria: str = Field(default='general', regex='^[a-z]+$')
    tags: List[str] = Field(default=[], max_items=5)
    
    # Validador personalizado
    @validator('nombre')
    def nombre_mayuscula(cls, v):
        return v.title()

# Producto valido
producto = Producto(nombre='laptop', precio=999.99, stock=10)
print(f'Producto: {producto}')

# Producto invalido
try:
    Producto(nombre='PC', precio=-100, stock=-5)
except Exception as e:
    print(f'\n❌ Error: {e}')

---

## 🏗️ Modelos Anidados

In [ ]:
# ============================================
# MODELOS ANIDADOS
# ============================================

class Direccion(BaseModel):
    calle: str
    ciudad: str
    codigo_postal: str
    pais: str = 'Colombia'

class Cliente(BaseModel):
    id: int
    nombre: str
    email: str
    direccion: Direccion  # Modelo anidado
    direcciones_envio: List[Direccion] = []

# Crear cliente con direccion
cliente = Cliente(
    id=1,
    nombre='Ana Garcia',
    email='ana@test.com',
    direccion=Direccion(
        calle='Calle 123',
        ciudad='Medellin',
        codigo_postal='050001'
    ),
    direcciones_envio=[
        Direccion(calle='Oficina', ciudad='Bogota', codigo_postal='110001')
    ]
)

print(f'Cliente: {cliente.json(indent=2)}')

---

## 📝 Schemas para FastAPI

In [ ]:
# ============================================
# SCHEMAS PARA FASTAPI
# ============================================

FASTAPI_SCHEMAS = '''
from pydantic import BaseModel, Field
from typing import Optional, List
from datetime import datetime

# Schema base (campos comunes)
class TareaBase(BaseModel):
    titulo: str = Field(..., min_length=1, max_length=100)
    descripcion: Optional[str] = Field(None, max_length=500)
    prioridad: int = Field(default=2, ge=1, le=5)

# Schema para crear (sin ID)
class TareaCreate(TareaBase):
    pass

# Schema para actualizar (todos opcionales)
class TareaUpdate(BaseModel):
    titulo: Optional[str] = Field(None, min_length=1, max_length=100)
    descripcion: Optional[str] = Field(None, max_length=500)
    prioridad: Optional[int] = Field(None, ge=1, le=5)
    estado: Optional[str] = Field(None, regex='^(pendiente|en_progreso|completada)$')

# Schema para respuesta (con ID y metadatos)
class TareaResponse(TareaBase):
    id: int
    estado: str
    creado_en: datetime
    actualizado_en: datetime
    
    class Config:
        orm_mode = True  # Permite convertir desde ORM

# Uso en FastAPI
from fastapi import FastAPI

app = FastAPI()

@app.post("/tareas", response_model=TareaResponse)
async def create_tarea(tarea: TareaCreate):
    # Validacion automatica!
    return {"id": 1, "estado": "pendiente", **tarea.dict(), ...}

@app.patch("/tareas/{id}", response_model=TareaResponse)
async def update_tarea(id: int, tarea: TareaUpdate):
    # Solo actualiza campos proporcionados
    return {"id": id, ...}
'''

print(FASTAPI_SCHEMAS)

---

**¡Valida todo, confia en nada! ✅**